In [ ]:
import os

import numpy as np
import pandas as pd

from utils import load_clean_surface_data
from utils import compute_power_density
from utils import estimate_normals

# Surface

In [ ]:
# compute power density and normal distributions for each exposure scenario
# create an augmented dataset

path_processed = os.path.join('data', 'surface', 'processed')
antennas = ['DipoleVertical', 'DipoleHorizontal',
            'ArrayVertical', 'ArrayHorizontal']
distances = [5, 10, 15]
for antenna in antennas:
    for distance in distances:
        df = load_clean_surface_data(antenna, distance)
        xyz = df[['x', 'y', 'z']].to_numpy()
        n = estimate_normals(xyz)
        E = tuple(component for component
                  in df[['Ex', 'Ey', 'Ez']].astype('complex128').to_numpy().T)
        H = tuple(component for component
                  in df[['Hx', 'Hy', 'Hz']].astype('complex128').to_numpy().T)
        S = compute_power_density(E, H)
        PD = np.abs(S[0].real * n[:, 0] + S[1].real * n[:, 1] + S[2].real * n[:, 2])
        df['Sx'] = S[0]
        df['Sy'] = S[1]
        df['Sz'] = S[2]
        df['nx'] = n[:, 0]
        df['ny'] = n[:, 1]
        df['nz'] = n[:, 2]
        df['PD'] = PD
        # df.to_csv(os.path.join(path_processed, f'{antenna}_d{distance}mm.csv'))